# Exploring Terms in the Encyclopaedia Britannica

## Summarize text - Gensim - BERT- GPT2

In this notebook we are going to do summarize the text of terms definitions with the dataframe that we have obtained either with the posprocess_eb.py script or Merging_EB_Terms.ipynb notebooks. Both methods obtain the same dataframe. 

We have selected the first Edition for this explorations, but we can run this notebook with any of the other editions.

**Remark**: Edition 1, has 3 volumes, and it was printed twice, in 1771 and 1773. 

These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.

- **Extractive Summarization**: These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.

- **Abstractive Summarization**: These methods use advanced NLP techniques to generate an entirely new summary. Some parts of this summary may not even appear in the original text.
 
Here we are going to focus on **extractive summarization technique**, using two libraries:
- Gensim
- Spacy


### Loading the necessary libraries

In [1]:
import yaml
import matplotlib.pyplot as plt
import numpy as np
import collections
import matplotlib as mpl

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize

In [4]:
import gensim
from gensim.summarization.summarizer import summarize

In [5]:
from summarizer import Summarizer, TransformerSummarizer

/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/f

### Functions

In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [8]:
def create_document(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return doc

## We have dataframe with these columns

- definition:           Definition of a term
- editionNum:           1,2,3,4,5,6,7,8
- editionTitle:         Title of the edition
- header:               Header of the page's term                                  
- place:                Place where the volume was edited (e.g. Edinburgh)                                    
- relatedTerms:         Related terms (see X article)  
- altoXML:              File Path of the XML file from which the term belongs       
- term:                 Term name                            
- positionPage:         Position of ther term in the page     
- startsAt:             Number page in which the term definition starts 
- endsAt:               Number page in which the term definition ends 
- volumeTitle:          Title of the Volume
- typeTerm:             Type of term [Topic| Articles]                                       
- year:                 Year of the edition
- volumeNum:            Volume number (e.g. 1)
- letters:              leters of the volume (A-B)
- part:                 Part of the volume (e.g 1)
- supplement:           Supplement's Title
- supplementsTo:        It suppelements to editions [1, 2, 3....]
- numberOfWords:        Number of words per term definition
- numberOfTerms:        Number of terms per page
- numberOfPages:        Number of pages per volume

### 1. Load dataframe from JSON file

### 3.2 Testing with Spacy

Following https://medium.com/analytics-vidhya/text-summarization-using-spacy-ca4867c6b744 

In [17]:
doc=create_document(text)

In [18]:
len(list(doc.sents))

298

#### 3.2.1 Filtering tokens

In [19]:
keyword =[]
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if (token.text in stopwords or token.text in punctuation):
        continue
    if (token.pos_ in pos_tag):
        keyword.append(token.text)

In [20]:
freq_word = Counter(keyword)
freq_word.most_common(5)

[('malt', 92), ('beer', 59), ('drink', 51), ('water', 49), ('time', 47)]

#### 3.2.2 Normalization 

In [21]:
max_freq = Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)
freq_word.most_common(5)

[('malt', 1.0),
 ('beer', 0.6413043478260869),
 ('drink', 0.5543478260869565),
 ('water', 0.532608695652174),
 ('time', 0.5108695652173914)]

#### 3.2.3 Weighing sentences

In [22]:
sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent]+=freq_word[word.text]
            else:
                sent_strength[sent] = freq_word[word.text]
#print(sent_strength)

#### 3.2.4 Summarizing the string

And the nlargest function returns a list containing the top 3 sentences which are stored as summarized_sentences

In [23]:
summarized_sentences = nlargest(3, sent_strength, key=sent_strength.get)


In [24]:
final_sentences = [w.text for w in summarized_sentences]
summary =' '.join(final_sentences)
print(summary)

In boiling, both time and the curdling or breaking of the wort should be consulted ; for if a person was to boil the wort an hour, and then take it out of the copper before it was rightly broke, it would be wrong management, and the drink would not be fine and wholesome; and if it Ihould boil an hour and a half, or two hours, without regarding when its particles are in a right order, then it may be too thick; so that due care must be had to the two extremes, to obtain it in its due order; therefore, in Odtober and keeping beers, an hour and a quarter’s good boiling is commonly sufficient to have a thorough cured drink; for generally in that time it will break and boil enough ; because in this there is a double security by length of boiling, and a quantity of hops shifted ; but in the new way there is only a single one, and that is by a double or treble allowance of fresh hops boiled only half an hour in the wort; and for this practice a reason is afligned, that the hops, being endowed 

In [29]:
len(summary)

4036

### 3.3 BERT

In [25]:
model = Summarizer()
result = model(text, min_length=60)
summary = "".join(result)
print(summary)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Whett feeds are thus sufficiently malted, they mult be dried in malt-kilns, the fuel of which ftiould smoke as little as poflible.—The hulks mull now be broke open by malt-milns, and then infused or riialh’ed nn warm water, in order to extradt the faccharine substance; the heat applied Ihould be very slow and gradual. Thus the malt is diflblved, and lies till the liquor be sufficiently tindiured. Here it must lie five or six tides, as the rbaltller calls it, reckoning twelve hours to the tide, according as the barley is in body or in driness. The way to know when it is enough, is to take a corn, end-ways. But by many the hair cloth is reckoned the best. some wrongly called choak, and rightly named culm or Welch-coal, from Swanzey in Pembrokeshire, being of a hard stony substance, in small bits, resembling a fhinning coal, and will burn without smoke, and by its fulphureous effluvia call a most excellent whiteness on all the outward parts of the grainy body: In Devonfflire their marble

### 3.4 GPT2

In [27]:

GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(text, min_length=60))
print(full)


Whett feeds are thus sufficiently malted, they mult be dried in malt-kilns, the fuel of which ftiould smoke as little as poflible.—The hulks mull now be broke open by malt-milns, and then infused or riialh’ed nn warm water, in order to extradt the faccharine substance; the heat applied Ihould be very slow and gradual. This tindlure obtained from the infusion of grinded malt, is commonly known by the name ok onort. But by many the hair cloth is reckoned the best. To know good from bad Malts. board, it will mark, and cast a white colour almost like chalk. -The pale malt is the slowest and flackeft dried of any, and where it has had a leisure fire, a sufficient time allowed it on the kiln, and a due care taken of it, the flour of the grain will remain in its full quantity, and thereby produce a greater length of wort than the brown high-dried malt. In the country they frequently throw a sack of malt on a stone or brick floor as soon as it is ground, and let it lie, giving it one turn, fo

### 3.5 XLNET

In [ ]:
model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
full = ''.join(model(text, min_length=60))
print(full)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
